In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchinfo

In [2]:
path_and_labels = pd.read_csv('train.csv')

In [3]:
path_and_labels.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [4]:
print(path_and_labels['participant_id'].value_counts())

participant_id
49445    4968
61333    4900
36257    4896
16069    4848
26734    4841
55372    4826
2044     4810
37779    4782
32319    4753
29302    4722
22343    4677
53618    4656
37055    4648
28656    4563
62590    4563
34503    4545
27610    4275
25571    3865
18796    3502
4718     3499
30680    3338
Name: count, dtype: int64


Participant это скорее всего отдельные люди с которых собирались жесты. Для нас тут важно лишь то что данные не с одного человека и даже не с нескольких, тут данные с нескольких десятков участников, наша модель не будет учитывать особенности отдельных людей, более нас ничего в participant пока не интересует. 

In [5]:
k = 0
data = pd.read_parquet(path_and_labels.values[k, 0])
data.head()

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152


In [6]:
len(data[data.frame == 20])

543

In [ ]:
data[data.frame == 20].type.value_counts()

type
face          468
pose           33
left_hand      21
right_hand     21
Name: count, dtype: int64

In [8]:
face_points = [0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55,
               61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107,
               133, 145, 146, 148, 149, 159, 178, 185, 191, 195,
               199, 205, 263, 267, 270, 276, 282, 285, 291, 295,
               308, 311, 312, 314, 317, 321, 324, 330, 331, 336,
               362, 374, 375, 377, 378, 386, 402, 409, 415, 425]

print(face_points)
print()
points = [*face_points, *range(468, 543)]
print(points)
print(len(points))

[0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55, 61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107, 133, 145, 146, 148, 149, 159, 178, 185, 191, 195, 199, 205, 263, 267, 270, 276, 282, 285, 291, 295, 308, 311, 312, 314, 317, 321, 324, 330, 331, 336, 362, 374, 375, 377, 378, 386, 402, 409, 415, 425]

[0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55, 61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107, 133, 145, 146, 148, 149, 159, 178, 185, 191, 195, 199, 205, 263, 267, 270, 276, 282, 285, 291, 295, 308, 311, 312, 314, 317, 321, 324, 330, 331, 336, 362, 374, 375, 377, 378, 386, 402, 409, 415, 425, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
140


In [9]:
k = 0

data = pd.read_parquet(path_and_labels.values[k, 0])
A = data.loc[data.frame == data.frame.value_counts().index[0],["x", "y", "z"]]

A = A.to_numpy().reshape(1, 543, 3)
A = torch.tensor(A, dtype = torch.float32)
print(A.shape)

A = A[:, points]
print(A.shape)

A = A.reshape(1, -1)
print(A.shape)

torch.Size([1, 543, 3])
torch.Size([1, 140, 3])
torch.Size([1, 420])


In [10]:
k = 0

data = pd.read_parquet(path_and_labels.values[k, 0])
B = data.loc[:,["x", "y", "z"]]

B = B.to_numpy().reshape(len(data)//543, 543, 3)
B = torch.tensor(B, dtype=torch.float32)
print(B.shape)

B = B[:, points]
print(B.shape)

B = B.reshape(B.shape[0], -1)
print(B.shape)

torch.Size([23, 543, 3])
torch.Size([23, 140, 3])
torch.Size([23, 420])


In [11]:
class Tokenizer(nn.Module):
    def __init__(self, embedding_dim):
        super(Tokenizer, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = 512
        
        self.max_len_seq = 500
        
        self.FF = nn.Sequential(
            nn.Linear(420, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.embedding_dim))
            
        self.position_encoding = self._create_position_encoding(self.embedding_dim, self.max_len_seq)
        
    def forward(self, x):
        #x.shape == [0:self.max_seq, 1024] 
        x = torch.nan_to_num(x, nan=0)  # УБРАТЬ ПОСЛЕ ТОГО КАК ПОДГОТОВИМ ДАННЫЕ ДЛЯ ОБУЧЕНИЯ
        x = self.FF(x)
        x = x + self.position_encoding[x.size(0), :]
        return x
        
    def _create_position_encoding(self, embedding_dim, max_seq_len):
        position_encoding = torch.zeros(max_seq_len, embedding_dim)
        for pos in range(max_seq_len):
            for i in range(0, embedding_dim, 2):
                position_encoding[pos, i] = np.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                position_encoding[pos, i+1] = np.cos(pos / (10000 ** ((2 * (i+1))/embedding_dim)))
        return position_encoding
    

In [12]:
token = Tokenizer(512)

In [13]:
token.position_encoding.shape

torch.Size([500, 512])

In [14]:
token.FF

Sequential(
  (0): Linear(in_features=420, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
)

In [15]:
print(B.shape)
token(B).shape

torch.Size([23, 420])


torch.Size([23, 512])

In [16]:
torchinfo.summary(token)

Layer (type:depth-idx)                   Param #
Tokenizer                                --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       215,552
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       262,656
Total params: 478,208
Trainable params: 478,208
Non-trainable params: 0

In [107]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.d_model = 128
        self.out_hid_dim = 512
        self.num_MHA_layers = 8
        
        self.embeder = Tokenizer(self.d_model)
        
        self.TE = nn.TransformerEncoder(nn.TransformerEncoderLayer(self.d_model, nhead = 4, dim_feedforward=512), self.num_MHA_layers)
        
        self.outFF = nn.Sequential(
            nn.Linear(self.d_model, self.out_hid_dim),
            nn.ReLU(),
            nn.Linear(self.out_hid_dim, 250),
            nn.Softmax())
        
    def forward(self, x):
        x = self.embeder(x)
        x = self.TE(x)
        x = x.mean(dim=0)
        x = self.outFF(x)
        return x
        
    

In [108]:
trans = Transformer()

In [109]:
torchinfo.summary(trans)

Layer (type:depth-idx)                                            Param #
Transformer                                                       --
├─Tokenizer: 1-1                                                  --
│    └─Sequential: 2-1                                            --
│    │    └─Linear: 3-1                                           215,552
│    │    └─ReLU: 3-2                                             --
│    │    └─Linear: 3-3                                           65,664
├─TransformerEncoder: 1-2                                         --
│    └─ModuleList: 2-2                                            --
│    │    └─TransformerEncoderLayer: 3-4                          198,272
│    │    └─TransformerEncoderLayer: 3-5                          198,272
│    │    └─TransformerEncoderLayer: 3-6                          198,272
│    │    └─TransformerEncoderLayer: 3-7                          198,272
│    │    └─TransformerEncoderLayer: 3-8                          198

In [110]:
k = 1

data = pd.read_parquet(path_and_labels.values[k, 0])
B = data.loc[:,["x", "y", "z"]]

B = B.to_numpy().reshape(len(data)//543, 543, 3)
B = torch.tensor(B, dtype=torch.float32)
print(B.shape)

B = B[:, points]
print(B.shape)

B = B.reshape(B.shape[0], -1)
print(B.shape)

torch.Size([11, 543, 3])
torch.Size([11, 140, 3])
torch.Size([11, 420])


In [114]:
print(trans(B).shape)
print(torch.argmax(trans(B)))
print(trans(B)[torch.argmax(trans(B))])
print(torch.sum(trans(B)))

torch.Size([250])
tensor(192)
tensor(0.0078, grad_fn=<SelectBackward0>)
tensor(1., grad_fn=<SumBackward0>)


In [115]:
print(np.random.rand(10000, 10000) @ np.random.rand(10000, 10000))

[[2494.71486352 2522.20137069 2486.07244947 ... 2501.77147855
  2506.76663437 2495.93293173]
 [2495.16367673 2474.49547426 2472.80387299 ... 2486.7408616
  2483.58106683 2488.73319224]
 [2482.37054233 2507.74985623 2482.4590969  ... 2498.69524811
  2499.03465422 2510.49698171]
 ...
 [2505.87670669 2485.42382205 2480.86654236 ... 2488.45465917
  2470.72833679 2478.63945965]
 [2490.7092854  2493.22987929 2482.21846644 ... 2506.92615938
  2494.5033894  2488.37671063]
 [2501.50635743 2516.51164471 2484.95728774 ... 2498.46621787
  2491.26226584 2495.34031753]]
